In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Heat-Transfer Equation

import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd
import random
import numpy as np
import time
import scipy.io

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda':
    print(torch.cuda.get_device_name())



cpu


In [3]:

# !pip install pyDOE2

In [4]:
# Reading data
data = pd.read_csv('/content/gdrive/MyDrive/project_one/3D_Heattransfer/Temp_3d_steady.csv')
boundary = pd.read_csv('/content/gdrive/MyDrive/project_one/3D_Heattransfer/Temp_3d_steady_boundary.csv')
dirichlet = pd.read_csv('/content/gdrive/MyDrive/project_one/3D_Heattransfer/Temp_3d_steady_boundary_dirichlet.csv')


In [5]:

# Vertically stack the DataFrames
data = pd.concat([data, boundary], ignore_index=True)
data = pd.concat([data, dirichlet], ignore_index=True)

print(len(data))

2789


In [6]:
del_x = 15
x = data.loc[:, ['x', 'y', 'z']].to_numpy()
count_ = x.shape[0]
print(count_)

'distant algorithm'
B = []

for i in range (0,count_):
  B_0 = np.exp(-((x - x[i, :])/del_x)**2)
  B_0 = np.array(np.exp(-((x - x[i, :])/del_x)**2))
  B_0 = np.sum((B_0**2)/3, axis=1)
  B_0 = B_0.reshape((B_0.shape[0], 1))
  B_0 = B_0.reshape((1,-1))
  B.append(B_0)

# print(B)
print(B[0])
print(B[0].min())
print(B[0].mean())

2789
[[1.         0.64200983 0.6712906  ... 0.65005108 0.53966225 0.61978011]]
0.0005824689211268873
0.2652518093541732


In [7]:
# another distant scale function
del_x = 10
'distant algorithm'
B = []
for i in range (0,count_):
  B_0 = np.exp(-((x - x[i, :])/del_x)**2)
  B_0 = np.array(np.exp(-((x - x[i, :])/del_x)**2))
  B_0 = np.sum((B_0**2)/3, axis=1)
  B_0 = B_0.reshape((B_0.shape[0], 1))
  B_0 = B_0.reshape((1,-1))
  B.append(B_0)

print(B[0].min())
print(B[0].mean())

2.0809671930412085e-07
0.19739442280933775


In [8]:
T = data['T'].to_numpy()
print(T.shape)
T_max = T.max()
T_min = T.min()
T_mean = T.mean()
T = ((T-T_min)/(T_max-T_min))

# T = ((T-T_min)/(T_max-T_min))*2 - 1

(2789,)


In [9]:
T = T.flatten('F')[:,None] #Fortran style (Column Major)
print(T.shape)

'create a list having [B[i],T[i]]'

main_array = np.arange(count_)

random_numbers = np.random.choice(main_array, size=int(count_ * 0.15), replace=False)

# Remove the generated numbers from the main array
main_array_after_removal = np.setdiff1d(main_array, random_numbers)

T_train_nu = []
x_train_nu = []
x_train_nf = []
for i in random_numbers:
  T_train_nu.append(T[i])
  x_train_nu.append(B[i])

for j in main_array_after_removal:
  x_train_nf.append(B[j])


print(T_train_nu[:2])
print(x_train_nu[:2])

# x_train_nu0 = np.vstack(x_train_nu)
# print(x_train_nu0[:2])

(2789, 1)
[array([0.0523957]), array([0.03789113])]
[array([[3.22577744e-01, 1.94674932e-04, 8.76469013e-05, ...,
        6.32292014e-06, 5.07665992e-09, 1.11820876e-04]]), array([[0.13484048, 0.26678194, 0.14926722, ..., 0.13640465, 0.33333333,
        0.04511176]])]


In [10]:
T_train_nu = np.vstack(T_train_nu)
x_train_nu = np.vstack(x_train_nu)
x_train_nf = np.vstack(x_train_nf)
B = np.vstack(B)

print(x_train_nu[:2])
print(T_train_nu[:2])
print(B[:2])

[[3.22577744e-01 1.94674932e-04 8.76469013e-05 ... 6.32292014e-06
  5.07665992e-09 1.11820876e-04]
 [1.34840479e-01 2.66781942e-01 1.49267223e-01 ... 1.36404655e-01
  3.33333334e-01 4.51117613e-02]]
[[0.0523957 ]
 [0.03789113]]
[[1.         0.5982438  0.66546451 ... 0.63044065 0.43196012 0.56616835]
 [0.5982438  1.         0.93741614 ... 0.56114869 0.56150496 0.43189448]]


In [11]:
'Convert to tensor and send to GPU'
print(x_train_nf.shape[1])
print(x_train_nu.shape)
print(T_train_nu.shape)
print(B.shape)
print(T.shape)
x_train_nf = torch.from_numpy(x_train_nf).float().to(device)
x_train_nu = torch.from_numpy(x_train_nu).float().to(device)
T_train_nu = torch.from_numpy(T_train_nu).float().to(device)
x_test = torch.from_numpy(B).float().to(device)
T_true = torch.from_numpy(T).float().to(device)
f_hat = torch.zeros(x_train_nf.shape[0],1).to(device)

2789
(418, 2789)
(418, 1)
(2789, 2789)
(2789, 1)


In [12]:
k = 45
lambda_u = 1
lambda_f = 0.5e-8
# lambda_f = 0.5e-5

class SwishActivation(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class FCN(nn.Module):

    def __init__(self,layers,activations):
        super().__init__() #call __init__ from parent class

        # class SwishActivation(nn.Module):
        #     def forward(self, x):
        #          return x * torch.sigmoid(x)

        # assert len(layers) - 1 == len(activations), "Number of activations should match the number of layers."

        'activation function'
        # self.activation = SwishActivation()
        self.activations = nn.ModuleList(activations)

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')

        'Initialise neural network as a list using nn.Modulelist'
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])

        self.iter = 0

        'Xavier Normal Initialization'
        for i in range(len(layers)-2):

            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)

            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)

    'foward pass'
    def forward(self,x):

        #convert to float
        a = x.float()

        # for i in range(len(layers)-1):

        #     z = self.linears[i](a)

        #     a = self.activation(z)

        # a = self.linears[-1](a)

        for i in range(len(self.linears) - 1):
            z = self.linears[i](a)
            a = self.activations[i](z)

        a = self.linears[-1](a)

        return a

    def loss_space(self,x,y):

        loss_T = self.loss_function(self.forward(x), y)

        return loss_T

    def loss_PDE(self, x_train_nf):

        g = x_train_nf.clone()

        g.requires_grad = True

        T_nfp = self.forward(g)

        T_nfp_x_y_z = autograd.grad(T_nfp,g,torch.ones([x_train_nf.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]

        T_nfp_xx_yy_zz = autograd.grad(T_nfp_x_y_z,g,torch.ones(x_train_nf.shape).to(device), create_graph=True)[0]

        T_nfp_xx = T_nfp_xx_yy_zz[:,[0]]
        T_nfp_yy = T_nfp_xx_yy_zz[:,[1]]
        T_nfp_zz = T_nfp_xx_yy_zz[:,[2]]

        #f = u_t + (self.forward(g))*(u_x) - (nu)*u_xx
        f = k*(T_nfp_xx + T_nfp_yy + T_nfp_zz)
        loss_f = self.loss_function(f,f_hat)

        return loss_f

    def loss(self,x,y,x_train_nf):

        loss_u = self.loss_space(x,y)
        loss_f = self.loss_PDE(x_train_nf)
        # print('loss_u')
        # print(loss_u.mean())
        # print('loss_f')
        # print(loss_f.mean())

        # loss_val = (lambda_u * loss_u) + loss_f
        # loss_val = loss_u + loss_f*lambda_f

        # loss_val = (lambda_u * loss_u ) + (loss_f * lambda_f * loss_f.mean())

        loss_val = (lambda_u * loss_u ) + (loss_f * lambda_f)

        return loss_val

    'callable for optimizer'
    def closure(self):

        loss = self.loss(x_train_nu, T_train_nu, x_train_nf)

        loss.backward()

        self.iter += 1

        if self.iter % 10 == 0:

            error_vec, _ = PINN.test()

            # print(loss,error_vec)
            print(f"Epoch {epoch+1}: loss = {loss:.8f}, error percent = {error_vec*100:.3f} %")

        return loss

    'test neural network'
    def test(self):

        u_pred = self.forward(x_test)
        # print('u_pred')
        # print(u_pred)
        # print('T_true')
        # print(T_true)

        error_vec = torch.linalg.norm((T_true-u_pred),2)/torch.linalg.norm(T_true,2)        # Relative L2 Norm of the error (Vector)

        u_pred = u_pred.cpu().detach().numpy()

        u_pred = np.reshape(u_pred,(-1, 1),order='F')

        return error_vec, u_pred

In [14]:
# Train the model
N = x_train_nf.shape[1]
layers = np.array([N,int(N/45),int(N/81),int(N/125),int(N/125),int(N/125),int(N/125),1]) #7 hidden layers (20)
activations = [SwishActivation(), SwishActivation(), SwishActivation(), SwishActivation(), SwishActivation(), SwishActivation(), nn.ReLU()]
PINN = FCN(layers, activations)

PINN.to(device)

# 'Neural Network Summary'
# print(PINN)

# params = list(PINN.parameters())

'''Optimization'''

'L-BFGS Optimizer'
# optimizer = torch.optim.LBFGS(PINN.parameters(), 1e-2,
#                               max_iter = 100,
#                               max_eval = None,
#                               tolerance_grad = 1e-10,
#                               tolerance_change = 1e-10,
#                               history_size = 100,
#                               line_search_fn = 'strong_wolfe')

# optimizer = torch.optim.LBFGS(PINN.parameters(), 1e-2,
#                               max_iter = 1000,
#                               max_eval = None,
#                               )

############################################################################
############################################################################


def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.kaiming_normal_(m.weight)

# use the modules apply function to recursively apply the initialization
PINN.apply(init_normal)

# optimizer = optim.Adam(PINN.parameters(), lr=8e-4, betas = (0.9,0.99),eps = 10**-15)

# For the first 100 epochs, use Adam optimizer
optimizer_adam = optim.Adam(PINN.parameters(), lr=8e-4, betas=(0.9, 0.99), eps=1e-15)
optimizer_lbfgs = optim.LBFGS(PINN.parameters(), lr=0.8)  # epoch < 100
# optimizer_adam = optim.Adam(PINN.parameters(), lr=2e-3, betas=(0.9, 0.99), eps=1e-15)
# optimizer_lbfgs = optim.LBFGS(PINN.parameters(), lr=0.9)    # epoch < 150

start_time = time.time()

for epoch in range(450):

  if 25 <= epoch < 50:
    lambda_f = 5e-6

  if 50 <= epoch <= 75:
    lambda_f = 5e-6 * 533.33

  if 125 <= epoch <= 200:
    lambda_f = 5e-6 * 533.33 * 5

  def Closure():
        optimizer_adam.zero_grad()
        loss = PINN.closure()
        return loss

  # Use Adam for the first 100 epochs
  if epoch < 105:   # 100 was better for unchanged lambdas   # 250 is ok for changed kambdas
      optimizer_adam.step(Closure)
  else:
      # Switch to L-BFGS for the remaining epochs
      optimizer_lbfgs.step(Closure)

  # PINN.zero_grad()
  # loss = PINN.closure()
  # optimizer.step()

############################################################################
############################################################################


# optimizer.step(PINN.closure)


elapsed = time.time() - start_time
print('Training time: %.2f' % (elapsed))


''' Model Accuracy '''
error_vec, u_pred = PINN.test()

print('Test Error: %.5f'  % (error_vec))


Epoch 10: loss = 0.06556849, error percent = 59.278 %
Epoch 20: loss = 0.04098829, error percent = 46.096 %
Epoch 30: loss = 0.02981420, error percent = 40.426 %
Epoch 40: loss = 0.02072389, error percent = 34.903 %
Epoch 50: loss = 0.01356841, error percent = 28.681 %
Epoch 60: loss = 0.01077138, error percent = 24.411 %
Epoch 70: loss = 0.00777927, error percent = 21.653 %
Epoch 80: loss = 0.00560113, error percent = 19.414 %
Epoch 90: loss = 0.00444794, error percent = 17.971 %
Epoch 100: loss = 0.00357441, error percent = 16.552 %
Epoch 106: loss = 0.00314677, error percent = 15.769 %
Epoch 106: loss = 0.00269687, error percent = 14.690 %
Epoch 107: loss = 0.00176471, error percent = 12.943 %
Epoch 107: loss = 0.00126528, error percent = 11.714 %
Epoch 108: loss = 0.00093819, error percent = 10.730 %
Epoch 108: loss = 0.00066597, error percent = 9.602 %
Epoch 109: loss = 0.00046264, error percent = 8.777 %
Epoch 109: loss = 0.00035339, error percent = 8.162 %
Epoch 110: loss = 0.00

In [22]:
T = data['T'].to_numpy()
print(T.shape)
T_max = T.max()
T_min = T.min()

print(u_pred.max())
print(T.max())
# T = ((T-T_min)/(T_max-T_min))


(2789,)
6392731.5
418.72


In [17]:
print(x.shape)
print(u_pred.shape)

u_pred = (u_pred * (T_max-T_min)) + T_min
result = np.hstack((x,u_pred.T))

print(result.shape)

(2789, 3)
(2789, 1)
(2789, 4)


In [19]:
# Specify the file path where you want to save the CSV file
file_path = '/content/gdrive/MyDrive/project_one/3D_Heattransfer/PINN_heat3D/my_array3.csv'

# Save the array as a CSV file
np.savetxt(file_path, result, delimiter=',')

In [ ]:
# print(u_pred.shape)

output_u = (u_pred*)

# output_u = (((u_pred+1)/2)*(T_max-T_min))+T_min

print(output_u)

x = x_test.detach().numpy()
x0 = x[:,0]
y0 = x[:,1]

# print(x0.shape)
# print(output_u.shape)

x_train_nu0 = x_train_nu.detach().numpy()
T_train_nu0 = T_train_nu.detach().numpy()
# print(x_train_nu[:,0].reshape(-1,1))
# print(T_train_nu0.shape)


plt.figure(figsize=(7,7))
plt.subplot(2, 1, 1)
plt.scatter(y0, x0, c = output_u , cmap = 'rainbow')
plt.title('Predicted Temperature of NN results')
plt.colorbar()
plt.show()

x1 = x0.reshape(33,33)
y1 = y0.reshape(33,33)

plt.subplot(1, 1, 1)
cp = plt.contourf(y1, x1, output_u,20,cmap="rainbow")
plt.colorbar(cp) # Add a colorbar to a plot
plt.scatter(x_train_nu[:,0].reshape(-1,1), x_train_nu[:,1].reshape(-1,1), c = T_train_nu0*0, marker='+')
plt.title('Predicted Temperature of NN results')
plt.xlabel('x')
plt.ylabel('y')

In [ ]:
# # save model
# torch.save(PINN.state_dict(),'/content/gdrive/MyDrive/project_one/2D_plate_Heattransfer/PINN_model3.pt')